<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=aa03714d54c3b105bcfca75dc34cb3c2797b27312e76fd214c5cca528255125b
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-09 13:06:47
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 25.40 K (0.18%)
Current PnL: -20.77 L (-13.78%)
CY Booked + Current PnL: -9.33 L (-6.19%)
-------------------
Total profit:  1.58 L
Total loss:  -22.35 L
-------------------
Total Booked + Current PnL: 17.65 L (14.27%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.12%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.27 L (62.69%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.62,121089.0,20585.0,8525.0,2.17,20.48,7.04,28.96,79.0,2.41,0.86,61.41,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.38,39.0,M-SC,1.00,83329.0,-17448.0,17541.0,0.13,-17.31,21.05,0.09,245.0,-0.99,0.59,9.48,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.03,188500.0,23830.0,19491.0,-0.39,14.47,10.34,26.31,89.0,1.22,1.34,23.61,X40N,NTT,AC
19,CIPLA,1662.74,-22.18,45.0,H-LC,3.67,207267.0,2767.0,20519.0,1.22,1.35,9.90,11.39,10.0,0.13,1.48,10.43,X40N,ATH,PHARMA
84,VOLTAS,1530.00,-2.20,48.0,H-MC,1.29,205095.0,13353.0,24406.0,-0.36,6.96,11.90,19.69,99.0,0.55,1.46,14.23,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SONACOMS,1006.00,-40.79,48.0,M-SC,13.75,73771.0,-27388.0,102276.0,4.02,-27.07,138.64,74.03,202.0,-0.27,0.53,4.02,AR,BTT,AUTO
55,RAJOOENG,143.10,-40.28,46.0,H-SC,16.40,90390.0,-12110.0,52706.0,4.02,-11.81,58.31,39.61,114.0,-0.23,0.64,10.39,AR,ATH,MISC
32,HINDZINC,730.22,35.55,73.0,M-LC,7.34,220784.0,15708.0,96858.0,3.48,7.66,43.87,54.89,52.0,0.16,1.57,33.60,X5K,ATH,METALS
21,COFFEEDAY,80.00,-45.67,48.0,L-SC,29.38,80640.0,-32909.0,72963.0,3.27,-28.98,90.48,35.27,268.0,-0.45,0.57,97.37,XR,NTT,HOTELS
69,SURYODAY,240.00,64.47,64.0,H-SC,8.11,151144.0,-27927.0,93664.0,2.48,-15.60,61.97,36.71,135.0,-0.30,1.08,49.96,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-35.01,43.0,M-SC,4.41,97234.0,-31436.0,76737.0,-2.46,-24.43,78.92,35.20,219.0,-0.41,0.69,11.60,X40,NTT,FOOTWEAR
73,TATAMOTORS,1065.00,-21.50,44.0,M-LC,2.10,247403.0,-29120.0,139189.0,-2.36,-10.53,56.26,39.81,54.0,-0.21,1.76,18.57,XY24,NTT,AUTO
18,CERA,9475.00,-23.87,36.0,H-SC,1.68,138908.0,-36995.0,79011.0,-2.23,-21.03,56.88,23.89,149.0,-0.47,0.99,19.97,OX40N,NTT,CERAMICS
41,IRCTC,1269.00,-35.98,40.0,H-MC,3.82,229227.0,-41281.0,184459.0,-1.92,-15.26,80.47,52.93,83.0,-0.22,1.63,4.94,XY24,BTT,TRAVEL
52,PGHH,17907.65,-27.37,56.0,H-MC,2.04,210240.0,9420.0,58384.0,-1.90,4.69,27.77,33.76,80.0,0.16,1.50,10.13,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,61.0,H-SC,13.53,130086.0,-12393.0,31910.0,1.01,-8.70,24.53,13.70,163.0,-0.39,0.93,53.54,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.38,39.0,M-SC,1.00,83329.0,-17448.0,17541.0,0.13,-17.31,21.05,0.09,245.0,-0.99,0.59,9.48,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,47.0,H-SC,1.97,220572.0,-49095.0,85428.0,0.20,-18.21,38.73,13.47,138.0,-0.57,1.57,12.15,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-22.53,52.0,H-MC,6.53,104899.0,-26936.0,69160.0,1.12,-20.43,65.93,32.03,98.0,-0.39,0.75,17.50,OX40N,NTT,IT
18,CERA,9475.0,-23.87,36.0,H-SC,1.68,138908.0,-36995.0,79011.0,-2.23,-21.03,56.88,23.89,149.0,-0.47,0.99,19.97,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.62,121089.0,20585.0,8525.0,2.17,20.48,7.04,28.96,79.0,2.41,0.86,61.41,MH,ATH,METALS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,46.75,56.0,M-MC,12.57,231127.0,6165.0,161488.0,1.90,2.74,69.87,74.53,192.0,0.04,1.65,35.88,XY24,BTT,STEEL
32,HINDZINC,730.22,35.55,73.0,M-LC,7.34,220784.0,15708.0,96858.0,3.48,7.66,43.87,54.89,52.0,0.16,1.57,33.60,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.03,188500.0,23830.0,19491.0,-0.39,14.47,10.34,26.31,89.0,1.22,1.34,23.61,X40N,NTT,AC
84,VOLTAS,1530.00,-2.20,48.0,H-MC,1.29,205095.0,13353.0,24406.0,-0.36,6.96,11.90,19.69,99.0,0.55,1.46,14.23,XY25,NTT,AC
17,CAMS,5211.76,-6.27,45.0,H-SC,1.65,106602.0,4598.0,39325.0,0.72,4.51,36.89,43.06,122.0,0.12,0.76,22.39,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.62,121089.0,20585.0,8525.0,2.17,20.48,7.04,28.96,79.0,2.41,0.86,61.41,MH,ATH,METALS
32,HINDZINC,730.22,35.55,73.0,M-LC,7.34,220784.0,15708.0,96858.0,3.48,7.66,43.87,54.89,52.0,0.16,1.57,33.60,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.50,193173.0,13365.0,99040.0,0.62,7.43,51.27,62.52,88.0,0.13,1.38,44.05,XR,NTT,BANKS
37,INDIAMART,4810.62,-54.08,45.0,H-SC,8.95,128011.0,4675.0,126949.0,-0.91,3.79,99.17,106.72,119.0,0.04,0.91,28.05,AR,ATH,MISC
86,WIPRO,420.00,-12.89,51.0,M-LC,5.65,152861.0,1916.0,107538.0,0.93,1.27,70.35,72.51,53.0,0.02,1.09,7.37,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.68,31.0,H-LC,5.63,205190.0,-46578.0,99784.0,0.14,-18.50,48.63,21.13,27.0,-0.47,1.46,10.37,X40,ATH,PAINTS
83,VBL,671.64,-22.30,33.0,H-LC,10.36,279604.0,-36238.0,148889.0,0.78,-11.47,53.25,35.67,5.0,-0.24,1.99,0.79,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,39.0,H-LC,7.53,245448.0,-16171.0,135389.0,-0.29,-6.18,55.16,45.57,7.0,-0.12,1.75,4.44,AR,NTT,ELECTRICAL
42,ITC,452.00,-39.61,40.0,H-LC,2.27,195478.0,-4660.0,25549.0,0.00,-2.33,13.07,10.44,4.0,-0.18,1.39,3.11,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-10.50,42.0,H-LC,8.02,251510.0,-5553.0,40694.0,0.60,-2.16,16.18,13.67,24.0,-0.14,1.79,17.81,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.17,53.0,H-LC,1.95,160840.0,-25255.0,72732.0,-1.05,-13.57,45.22,25.51,15.0,-0.35,1.15,18.53,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,40.0,H-LC,2.27,195478.0,-4660.0,25549.0,0.00,-2.33,13.07,10.44,4.0,-0.18,1.39,3.11,X40,NTT,FMCG
48,LTIM,7230.2,-2.02,61.0,H-LC,2.55,242685.0,-3466.0,82683.0,0.95,-1.41,34.07,32.18,16.0,-0.04,1.73,34.13,X200,ATH,IT
51,NESTLEIND,1377.0,-10.26,51.0,H-LC,2.61,276120.0,10694.0,46084.0,-0.04,4.03,16.69,21.40,11.0,0.23,1.97,10.94,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.79,43.0,H-LC,3.19,213314.0,2948.0,25832.0,-1.26,1.40,12.11,13.68,37.0,0.11,1.52,18.16,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,33.0,H-LC,10.36,279604.0,-36238.0,148889.0,0.78,-11.47,53.25,35.67,5.0,-0.24,1.99,0.79,X40N,ATH,FMCG
42,ITC,452.00,-39.61,40.0,H-LC,2.27,195478.0,-4660.0,25549.0,0.00,-2.33,13.07,10.44,4.0,-0.18,1.39,3.11,X40,NTT,FMCG
1,ABB,7934.00,-41.23,39.0,H-LC,7.53,245448.0,-16171.0,135389.0,-0.29,-6.18,55.16,45.57,7.0,-0.12,1.75,4.44,AR,NTT,ELECTRICAL
80,UNITDSPR,1644.00,-12.09,52.0,H-LC,6.66,230274.0,-3870.0,52502.0,0.43,-1.65,22.80,20.77,86.0,-0.07,1.64,5.15,X40N,NTT,BREWERIES
74,TCS,4406.34,-28.04,53.0,H-LC,12.16,286521.0,-59435.0,127674.0,0.69,-17.18,44.56,19.73,1.0,-0.47,2.04,5.53,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.17,53.0,H-LC,1.95,160840.0,-25255.0,72732.0,-1.05,-13.57,45.22,25.51,15.0,-0.35,1.15,18.53,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,40.0,H-LC,2.27,195478.0,-4660.0,25549.0,0.00,-2.33,13.07,10.44,4.0,-0.18,1.39,3.11,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.68,31.0,H-LC,5.63,205190.0,-46578.0,99784.0,0.14,-18.50,48.63,21.13,27.0,-0.47,1.46,10.37,X40,ATH,PAINTS
19,CIPLA,1662.74,-22.18,45.0,H-LC,3.67,207267.0,2767.0,20519.0,1.22,1.35,9.90,11.39,10.0,0.13,1.48,10.43,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-14.79,43.0,H-LC,3.19,213314.0,2948.0,25832.0,-1.26,1.40,12.11,13.68,37.0,0.11,1.52,18.16,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.04,53.0,H-LC,12.16,286521.0,-59435.0,127674.0,0.69,-17.18,44.56,19.73,1.0,-0.47,2.04,5.53,X40,ATH,IT
40,INFY,2275.00,-17.44,57.0,H-LC,7.90,321545.0,8657.0,163023.0,0.98,2.77,50.70,54.87,3.0,0.05,2.29,9.60,X40,BTT,IT
42,ITC,452.00,-39.61,40.0,H-LC,2.27,195478.0,-4660.0,25549.0,0.00,-2.33,13.07,10.44,4.0,-0.18,1.39,3.11,X40,NTT,FMCG
83,VBL,671.64,-22.30,33.0,H-LC,10.36,279604.0,-36238.0,148889.0,0.78,-11.47,53.25,35.67,5.0,-0.24,1.99,0.79,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,39.0,H-LC,7.53,245448.0,-16171.0,135389.0,-0.29,-6.18,55.16,45.57,7.0,-0.12,1.75,4.44,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,48.0,L-SC,29.38,80640.0,-32909.0,72963.0,3.27,-28.98,90.48,35.27,268.0,-0.45,0.57,97.37,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,39.0,H-SC,10.58,89820.0,-8160.0,29766.0,-1.51,-8.33,33.14,22.05,152.0,-0.27,0.64,30.56,XR,ATH,FINANCE
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.62,121089.0,20585.0,8525.0,2.17,20.48,7.04,28.96,79.0,2.41,0.86,61.41,MH,ATH,METALS
78,UJJIVANSFB,60.00,105.37,61.0,H-SC,13.53,130086.0,-12393.0,31910.0,1.01,-8.70,24.53,13.70,163.0,-0.39,0.93,53.54,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.39,145152.0,-4068.0,148244.0,1.38,-2.73,102.13,96.62,208.0,-0.03,1.03,57.09,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.39,145152.0,-4068.0,148244.0,1.38,-2.73,102.13,96.62,208.0,-0.03,1.03,57.09,XY25,NTT,FINANCE
32,HINDZINC,730.22,35.55,73.0,M-LC,7.34,220784.0,15708.0,96858.0,3.48,7.66,43.87,54.89,52.0,0.16,1.57,33.60,X5K,ATH,METALS
69,SURYODAY,240.00,64.47,64.0,H-SC,8.11,151144.0,-27927.0,93664.0,2.48,-15.60,61.97,36.71,135.0,-0.30,1.08,49.96,XR,NTT,BANKS
26,GREENPANEL,537.00,233.98,60.0,M-SC,4.91,149206.0,-33872.0,112844.0,-0.49,-18.50,75.63,43.14,230.0,-0.30,1.06,40.35,XY24,NTT,MISC
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.62,121089.0,20585.0,8525.0,2.17,20.48,7.04,28.96,79.0,2.41,0.86,61.41,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.15
1,25,43.44
2,50,75.12


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.91
LC    32.29
MC    23.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.92
X40      14.65
X40N     11.18
XR       10.93
XY25     10.36
AR        8.18
OX40N     7.67
X200      1.73
MH        1.61
X5K       1.57
SR        1.12
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.92
H-LC    25.74
H-MC    20.43
M-SC    13.57
M-LC     5.54
M-MC     2.99
L-SC     1.42
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.70,-6.76,41.94
IT,12.90,-16.16,79.08
FINANCE,10.46,-14.81,70.25
MISC,6.91,-18.29,83.83
BANKS,6.35,-12.12,71.09
PAINTS,5.50,-19.72,37.56
ELECTRICAL,5.33,-10.11,48.74
INSURANCE,3.83,-3.26,41.85
AUTO,3.53,-16.15,71.67


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3234018.0,22
XR,1375184.0,14
AR,1184127.0,9
X40,783770.0,10
X40N,665886.0,9
OX40N,570061.0,10
XY25,482871.0,7
SR,264884.0,2
X5K,96858.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3733779.0,29
M-SC,1523753.0,17
H-MC,1288792.0,15
H-LC,1180905.0,15
M-LC,372346.0,4
M-MC,364744.0,2
L-SC,260884.0,3
L-MC,60028.0,1
L-LC,42092.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1206567.0      6
M-SC       XY24       850423.0      7
H-SC       AR         808020.0      5
           XR         803538.0      7
H-MC       XY24       575655.0      4
H-LC       X40        486902.0      5
M-MC       XY24       364744.0      2
H-SC       X40N       307620.0      4
           SR         264884.0      2
           OX40N      264694.0      4
H-LC       X40N       221910.0      3
H-MC       X40        220131.0      4
H-LC       AR         208121.0      2
H-MC       XY25       179925.0      2
L-SC       XR         171738.0      2
M-SC       AR         167986.0      2
M-LC       XY24       167950.0      2
M-SC       XY25       148244.0      1
           OX40N      147061.0      4
H-MC       X40N       136356.0      2
M-SC       XR         133302.0      2
H-LC       XY25       112610.0      3
M-LC       XR         107538.0      1
H-MC       XR          99040.0      1
M-LC       X5K         96858.0      1
L-SC       OX40N       89146.0      1
H-LC       X200        82683.0      1
H-SC       MH          78456.0      1
M-SC       X40         76737.0      1
H-MC       OX40N       69160.0      1
H-LC       XY24        68679.0      1
L-MC       XR          60028.0      1
L-LC       XY25        42092.0      1
H-MC       MH           8525.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
